In [232]:
!pip install transformers
!pip install langdetect
!pip install langid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [233]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [234]:
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold

import re
import nltk
nltk.download('stopwords')

import gensim
import gensim.downloader as gensim_api

import transformers

import langdetect
import langid

import wget

import ast

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [235]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/213.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
ref = zipfile.ZipFile(model_file, 'r')
ref.extractall()

In [236]:
pd.options.display.max_colwidth = 150

In [237]:
tags = pd.read_csv("translated_preprocessed_tags.csv")
tags = list(tags["0"])
tags = ast.literal_eval(str(tags))
lst_dics = tags

In [238]:
dtf = pd.read_csv("translated_preprocessed_names.csv")
dtf["0"]

0                                             Новая концепция для музея «Микула» в Нижнем Новгороде
1                                          Развитие туристического кластера в Кроноцком заповеднике
2                            Создание мобильного приложения СППР для диагностики болезни Паркинсона
3                                                                      Гуманитарная политика России
4                                                        Подкаст про современное высшее образование
                                                   ...                                             
410                                Веб-приложение для генерации страницы-каталога услуг самозанятых
411    Распознавание и визуализация распространения лесных пожаров по данным со спутниковых снимков
412                                               Приложение-маркетплейс для продажи и обмена вещей
413                                                                            Бесплатный TON DeBot


In [239]:
langid.set_languages(['ru', 'en'])
lang, score = langid.classify(dtf["0"].iloc[1])
print(lang, score)

ru -1115.1721971035004


In [240]:
nlp = gensim.models.KeyedVectors.load("model.model")
nlp.most_similar(["психология"], topn=10)

[('психолингвистика', 0.7906066179275513),
 ('нейропсихология', 0.7846694588661194),
 ('парапсихология', 0.7698875665664673),
 ('педагогика', 0.7587850689888),
 ('социология', 0.7499883770942688),
 ('культурология', 0.7364685535430908),
 ('сексология', 0.7282429337501526),
 ('психофизиология', 0.717481255531311),
 ('конфликтология', 0.7139410376548767),
 ('лингвистика', 0.7136410474777222)]

In [241]:
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))


In [242]:
tokenizer = transformers.BertTokenizer.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', do_lower_case=True)

nlp = transformers.TFBertModel.from_pretrained('DeepPavlov/bert-base-multilingual-cased-sentence', from_pt=True)
#nlp = transformers.TFBertModel.from_pretrained('DeepPavlov/rubert-base-cased', from_pt=True)

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [243]:
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X

# Embedding the tags using BERT
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
                 for txt in lst_dics]


X = np.array(lst_mean_vecs)

In [244]:
# Embedding the titles using BERT
         
dic_y = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
  for txt in dtf['0']]

In [245]:
# Calculate cosine similarities
similarities = np.array([metrics.pairwise.cosine_similarity(X, y.reshape(1,-1)).T.tolist()[0] for y in dic_y]).T

In [246]:
labels = list(dtf['0'])

for i in range(len(similarities)):
    ### assign randomly if there is no similarity
    if sum(similarities[i]) == 0:
       similarities[i] = [0]*len(labels)
       similarities[i][np.random.choice(range(len(labels)))] = 1
    ### rescale so they sum = 1
    similarities[i] = similarities[i] / sum(similarities[i])

## classify the label with highest similarity score
predicted_prob = similarities
#predicted = [labels[np.argmax(pred)] for pred in predicted_prob]

In [247]:
a = np.array(labels)
predicted = [a[np.argpartition(pred, -4)[-4:]] for pred in predicted_prob]

In [249]:
fin = pd.DataFrame(predicted)

In [250]:
fin2 = pd.DataFrame(lst_dics)

In [251]:
result = pd.concat([fin2, fin], axis=1)

In [252]:
result

,0,0,1,2,3
0,"['арт', 'исследовательский проект', 'музейное дело', 'развитие регионов', 'туризм']","Мобильное приложение, агрегатор туристических маршрутов",Выдача рекламных предложений на основе геолокации (ВРПОГ),"Благотворительный фонд ""Точка Опоры"" – стратегия продвижения",Информационно-аналитический портал «Право благоприятной окружающей среды»
1,"['коммуникации', 'местные сообщества', 'развитие регионов', 'социологические исследования', 'туризм']",Информационная поддержка деятельности ПУЛАП в социальных сетях,"Мобильное приложение, агрегатор туристических маршрутов",Выдача рекламных предложений на основе геолокации (ВРПОГ),Информационно-аналитический портал «Право благоприятной окружающей среды»
2,"['IT-проекты', 'анализ рынка', 'веб-разработка', 'дизайн', 'мобильное приложение']",Приложение для продажи цифровых товаров и CRM-приложение для разработчиков,Разработка программного комплекса для мониторинга оборудования и автоматизации сервисного обслуживания,"Кроссплатформенное мобильное приложение - платформа для выкладывания курсов, постов, подкастов и проч.",Веб-приложение: виртуальный карьерный помощник
3,"['культура', 'международные отношения']",IOS-приложение для поиска и продвижения мероприятий,Скидка на доставку (ДД),Инструмент руководителя,Информационно-аналитический портал «Право благоприятной окружающей среды»
4,"['SMM', 'аудио', 'аудиоподкастинг']",Мобильное фитнес приложение,Программа для создания и редактирования составных медиа объектов,Программа для экспериментального исследования платформо-независимых средств видео-коммуникации,Веб-сервис обработки субтитр
5,"['миграция', 'полевые исследования', 'проведение интервью', 'экспедиция']","Программа для сбора, очистки и слияния геопространственных данных",Программа слияния спутниковых изображений с разным пространственным разрешением,IOS-приложение для поиска и продвижения мероприятий,"Мобильное приложение, агрегатор туристических маршрутов"
6,['социальные сети'],Инструмент руководителя,Скидка на доставку (ДД),Играем в диспетчера,PWA-Органайзер
7,"['открываем Россию заново', 'экспедиция']",Гуманитарная политика России,Дайджест экспертных мнений для Портала «Orientalia Rossica – российское востоковедение XXI века»,Разработка игры “SCP: The Siberian Incident”,"CRM-система для ""AliExpress Россия"""
8,['Китай'],Скидка на доставку (ДД),Зверон,Перлер,PWA-Органайзер
9,"['городское благоустройство', 'проектная деятельность', 'развитие регионов', 'экология']","Программа для сбора, очистки и слияния геопространственных данных",Развитие образовательного экологического центра на о. Кильпола (Карелия),"Благотворительный фонд ""Точка Опоры"" – стратегия продвижения",Информационно-аналитический портал «Право благоприятной окружающей среды»
